In [1]:
import gpkit
from gpkit import Model, Variable
from gpkit.constraints.tight import Tight
from gpkit.constraints.loose import Loose
from gpkit.constraints.bounded import Bounded
from gpkit import ureg

from prettytable import PrettyTable
import operator

from rocket import Rocket
from sankey3 import Sankey

import datetime

with gpkit.SignomialsEnabled():
    r = Rocket()
    prob = Model(r.m, Bounded(r))
    sol = prob.localsolve(iteration_limit=10)
    sol.datetime = datetime.datetime.now().strftime('%d-%m %H:%M')

print()
print(f'Solved at {sol.datetime}')        
print(sol.summary())

​Beginning signomial solve.
Solving took 5 GP solves and 1.54 seconds.

Solved at 05-11 19:18

Cost
----
 27.89 [kg]

Free Variables
--------------
               | Rocket
    a_{launch} : 86.54   [m/s²] Acceleration off launch rail
             m : 27.89   [kg]   Mass of Rocket
         min_a : 86.54   [m/s²] minimum launch acceleration

               | Rocket/Boosters
             F : 1687    [N]    Boosters cumulative thrust
             m : 0.9749  [kg]   Mass of Boosters
       m_{dry} : 0.6824  [kg]   Dry mass of boosters
      m_{prop} : 0.2925  [kg]   Propellant mass of boosters
      t_{burn} : 0.3467  [s]    Booster burn time

               | Rocket/SimpleEngine
     A_{grain} : 45.58   [cm²]  cross section area of grain
     L_{grain} : 0.176   [m]    Length of the grain
        L_{ox} : 0.4569  [m]    Length of ox tank
        V_{ox} : 8329    [cm³]  Volume of ox tank
             m : 13.82   [kg]   Mass of Engine
       m_{dry} : 7.68    [kg]   Dry mass of engine
      m

In [2]:

d=sol['sensitivities']['variables']
variables = sol['variables']
t = PrettyTable(['key','lineage','value','unit','sens', '-abs(sens)','descr'])
for key, val in d.items():
    if key in sol['constants'].keys():
        sens =  '%4.3f'%val
    else:
        sens = '*'
    
    t.add_row([key.name,key.lineagestr(),float(variables[key]),key.unitrepr,sens,-abs(float(val)),key.label])
    
    

t.align='r'
t.float_format='5.3'
t.reversesort = False
t.sortby = '-abs(sens)'

t.sort_key=operator.itemgetter(2,6)
print('SORTED BY LINEAGE, SENSITIVITY (solved on ' + sol.datetime + ')')
print(t.get_string(fields = ['key', 'lineage','value','unit','sens','descr']))


SORTED BY LINEAGE, SENSITIVITY (solved on 05-11 19:18)
+----------------+---------------------+----------+--------+--------+---------------------------------------+
|            key |             lineage |    value |   unit |   sens |                                 descr |
+----------------+---------------------+----------+--------+--------+---------------------------------------+
|            PMF |              Rocket |    0.220 |      - |  0.814 |     Propellant Mass Fraction required |
|     v_{launch} |              Rocket |   30.000 |    m/s |  0.125 |              Velocity off launch rail |
|     L_{launch} |              Rocket |    5.200 |      m | -0.029 |                 Length of launch rail |
|              g |              Rocket |    9.810 |  m/s^2 |  0.011 |           Acceleration due to gravity |
|     a_{launch} |              Rocket |   86.538 |  m/s^2 |      * |          Acceleration off launch rail |
|              m |              Rocket |   27.891 |     kg |     

In [3]:
t.sort_key=operator.itemgetter(6)
print('SORTED BY SENSITIVITY (solved on ' + sol.datetime + ')')
print(t.get_string(fields = ['key', 'lineage','value','unit','sens','descr']))

SORTED BY SENSITIVITY (solved on 05-11 19:18)
+----------------+---------------------+----------+--------+--------+---------------------------------------+
|            key |             lineage |    value |   unit |   sens |                                 descr |
+----------------+---------------------+----------+--------+--------+---------------------------------------+
|            PMF |              Rocket |    0.220 |      - |  0.814 |     Propellant Mass Fraction required |
| rho_{ox, tank} | Rocket/SimpleEngine | 8000.000 | kg/m^3 |  0.391 |            Density of ox tank (if al) |
|         Tank P | Rocket/SimpleEngine |   60.000 |    bar |  0.391 |                  Max Ox Tank pressure |
|             SF | Rocket/SimpleEngine |    3.000 |      - |  0.391 |          Wall thickness safety factor |
|   \sigma_{max} | Rocket/SimpleEngine |  585.000 |    MPa | -0.391 |        Max stress of tank, Al-7075-T6 |
|              m |   Rocket/Structures |    4.600 |     kg |  0.317 |     

In [12]:
(0.01048*ureg.m**3).to(ureg.cm**3)

10479.999999999998 <Unit('centimeter ** 3')>

In [4]:
Sankey(prob).diagram(r.m)

(objective) adds +1 to the sensitivity of m_Rocket
(objective) is m_Rocket [kg] 

Ⓐ adds -1.92 to the overall sensitivity of m_Rocket
Ⓐ is m_Rocket >= m_Rocket/Avionics + m_Rocket/Boosters + m_Rocket/Payload + m_Rocket/Recovery + m_Rocket/SimpleEngine + m_Rocket/Structures 

Ⓑ adds +0.814 to the overall sensitivity of m_Rocket
Ⓑ is m_{prop}_Rocket/SimpleEngine >= PMF_Rocket*m_Rocket 

Ⓒ adds +2.02e-12 to the overall sensitivity of m_Rocket
Ⓒ is m_Rocket <= 100 

Ⓓ adds -1.25e-11 to the overall sensitivity of m_Rocket
Ⓓ is m_Rocket >= 10 

Ⓔ adds +4.54e-12 to the overall sensitivity of m_Rocket
Ⓔ is F_Rocket/SimpleEngine >= TW_{main, min}_Rocket*g_Rocket*m_Rocket 

Ⓕ adds +1.04e-13 to the overall sensitivity of m_Rocket
Ⓕ is m_Rocket <= 1e+30 

Ⓖ adds -9.71e-14 to the overall sensitivity of m_Rocket
Ⓖ is m_Rocket >= 1e-30 



SankeyWidget(layout=Layout(height='400', width='900'), links=[{'source': 'm_Rocket', 'target': 'Model', 'value…

In [5]:
s = Sankey(prob)

In [6]:
s.sorted_by("constraints", 9)

Ⓐ adds +0.0671 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓐ is m_Rocket/Boosters >= m_{dry}_Rocket/Boosters + m_{prop}_Rocket/Boosters 

Ⓑ adds -1.65e-09 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓑ is m_{prop}_Rocket/Boosters >= 0.2 

Ⓒ adds -0.0671 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓒ is c_Rocket/Boosters*m_{prop}_Rocket/Boosters >= F_Rocket/Boosters*t_{burn}_Rocket/Boosters 

Ⓓ adds +8.65e-14 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓓ is m_{prop}_Rocket/Boosters <= 1e+30 

Ⓔ adds -1.17e-13 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓔ is m_{prop}_Rocket/Boosters >= 1e-30 



SankeyWidget(layout=Layout(height='400', width='900'), links=[{'source': 'm_{prop}_Rocket/Boosters', 'target':…

In [7]:
s.sorted_by("maxflow", 3)

Ⓐ adds +0.814 to the overall sensitivity of PMF_Rocket
Ⓐ is m_{prop}_Rocket/SimpleEngine >= PMF_Rocket*m_Rocket 



SankeyWidget(layout=Layout(height='400', width='900'), links=[{'source': 'PMF_Rocket', 'target': 'Model', 'val…

In [8]:
Sankey(prob).diagram(r.engine.m_ox)

Ⓐ adds +1.9 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓐ is m_{prop}_Rocket/SimpleEngine >= m_{fuel}_Rocket/SimpleEngine + m_{ox}_Rocket/SimpleEngine 

Ⓑ adds -2.18 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓑ is OF_Rocket/SimpleEngine**-1*m_{ox}_Rocket/SimpleEngine + m_{ox}_Rocket/SimpleEngine >= m_{prop}_Rocket/SimpleEngine 

Ⓒ adds +0.282 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓒ is V_{ox}_Rocket/SimpleEngine >= m_{ox}_Rocket/SimpleEngine*rho_{ox}_Rocket/SimpleEngine**-1 

Ⓓ adds +1.02e-13 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓓ is m_{ox}_Rocket/SimpleEngine <= 1e+30 

Ⓔ adds -9.94e-14 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓔ is m_{ox}_Rocket/SimpleEngine >= 1e-30 



SankeyWidget(layout=Layout(height='400', width='900'), links=[{'source': 'm_{ox}_Rocket/SimpleEngine', 'target…